In [39]:
#IMPORT statements
import pandas as pd

In [40]:
###### Drop output column - SalePrice #####
hp_df=pd.read_csv('house_price_data.csv')
y=hp_df['SalePrice']
hp_df.drop(columns='SalePrice',inplace=True)

In [41]:
#Get null values and Percentages
nainfo_df=pd.DataFrame({'col_name':hp_df.columns,
                        'na_count':hp_df.isnull().sum(),
                        'na_pc':hp_df.isnull().sum()/hp_df.shape[0]*100
                       })

In [42]:
# sort values based on null percentgae, Drop them
nainfo_df.sort_values(by='na_pc',ascending=False)
cols_grt50has=list(nainfo_df[nainfo_df['na_pc']>50]['col_name'])
#hp_df.drop(cols_grt50has,inplace=True)
cols_grt50has
#hp_df.drop(columns=cols_grt50has,inplace=True)


['Alley', 'PoolQC', 'Fence', 'MiscFeature']

In [43]:
hp_df.drop(columns='Id',inplace=True)   #delete un-necesssary columns


In [44]:
#Train Test Split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(hp_df,y,random_state=42,test_size=0.2)

In [45]:
# Fill Misiing Values

import warnings
warnings.filterwarnings('ignore')

hp_df['LotArea'].dtype
con_cols=[col for col in hp_df.columns if hp_df[col].dtype=='int64' or hp_df[col].dtype=='float64']
cat_cols=[col for col in hp_df.columns if hp_df[col].dtype=='object']

for col in con_cols:
    x_train[col].fillna(x_train[col].mean(),inplace=True)
    x_test[col].fillna(x_train[col].mean(),inplace=True)
    
for col in cat_cols:
    x_train[col].fillna(x_train[col].mode()[0],inplace=True)
    x_test[col].fillna(x_train[col].mode()[0],inplace=True)
    



In [46]:
# Scale continous variable

from sklearn.preprocessing import StandardScaler
import numpy as np

scaler=StandardScaler()

for col in con_cols:
    x_train[col]=scaler.fit_transform(np.array(x_train[col]).reshape(-1,1))
    x_test[col]=scaler.transform(np.array(x_test[col]).reshape(-1,1))

In [54]:
#Encode categorical variables

oe_train=pd.get_dummies(x_train[col])
oe_test=pd.get_dummies(x_test[col])

oe_test

,-1.375486,-0.619098,0.137290,0.893677,1.650065
892,1,0,0,0,0
1105,0,0,0,0,1
413,0,0,0,0,1
522,1,0,0,0,0
1036,0,0,0,1,0
...,...,...,...,...,...
479,0,1,0,0,0
1361,0,0,0,1,0
802,0,0,1,0,0
651,0,0,0,1,0


In [61]:
#align values 

oe_train_final,oe_test_final=oe_train.align(oe_test,join='inner',axis=1)


In [59]:
#concatinate Continous, Categorical 

x_train_final=pd.concat([x_train[con_cols],oe_train_final],axis=1)
x_test_final=pd.concat([x_test[con_cols],oe_test_final],axis=1)



In [79]:
# Build liner regression model

from sklearn.linear_model import LinearRegression
linreg=LinearRegression()

linreg.fit(x_train_final,y_train)
lin_sale_pred=linreg.predict(x_test_final)


In [80]:
#check r2 score

from sklearn.metrics import r2_score

r2_score(y_test,lin_sale_pred)
#r2_score(y_test,sale_pred)

0.8212441588378957

In [77]:
#Decision tree regresser

from sklearn.tree import DecisionTreeRegressor

dcstree=DecisionTreeRegressor()

dcstree.fit(x_train_final,y_train)
dcs_sale_pred=dcstree.predict(x_test_final)


In [78]:
#R2 score for decision tree model

r2_score(y_test,dcs_sale_pred)

0.8025852517882497